# B. Process from the XLSX files

- **3:citations** `input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/3_2025_March 9th_citation_counts.xlsm`
  -  il s’agit des citations : je ne l’ai pas travaillé.  
 - **4:first&last** `input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/4_working first and last.xlsx`
    -   ils’agit d’une liste de 20 auteurs qui sont first and last and mais ne pas prendre en compte ceux qui sont PI.  

In [1]:
import pandas as pd
import pickle
import preprocess_utils
import numpy as np
import json
import plot_info

def clean_authors_claims(df):
    df['Sex'] = df['Sex'].map({1: 'Male', 0: 'Female', '??': np.nan, '?': np.nan})
    df = df.drop(columns=[col for col in df.columns if '%' in col])
    df = df.drop(columns=[col for col in df.columns if 'Unnamed' in col])
    df = df.rename(columns={'Conituinity': 'Continuity', 
                                                "Partially verified":"Partially Verified", 
                                                "*Historical lab after 1998":"Historical lab after 1998"
                                                })
    if "Historical lab" in df.columns:
        df['Historical lab'] = df['Historical lab'].astype('boolean')
    if "Continuity" in df.columns:
        df['Continuity'] = df['Continuity'].astype('boolean')
    if "Become a Pi" in df.columns:
        df.loc[df["Become a Pi"] == "??", "Become a Pi"] = np.nan
        df.loc[df["Become a Pi"] == 0, "Become a Pi"] = False
        df.loc[df["Become a Pi"] == 1, "Become a Pi"] = True
    if "MD" in df.columns:
        df.loc[df["MD"] == "??", "MD"] = np.nan
        df.loc[df["MD"] == "?", "MD"] = np.nan
        df.loc[df["MD"] == 'O', "MD"] = False
        df.loc[df["MD"] == 0, "MD"] = False
        df.loc[df["MD"] == 1, "MD"] = True
    if "Ivy league" in df.columns:
        df.loc[df["Ivy league"] == 0, "Ivy league"] = False
        df.loc[df["Ivy league"] == 1, "Ivy league"] = True
    if "PhD Post-doc" in df.columns:
        df.loc[df["PhD Post-doc"] == "pi", "PhD Post-doc"] = "PI"
        df.loc[df["PhD Post-doc"] == "Pi", "PhD Post-doc"] = "PI"
    df = df.dropna(subset=['Name'])
    # drop  Schneider DS+D from names as  Schneider DS and  Schneider D are also there in separate rows
    df = df[~(df['Name'] == 'Schneider DS+D')]
    return df


# Load the database stuff
authors_db = pd.read_csv("preprocessed_data/authors_db.csv")
authors_db.sort_values("author_key")


claims_db = pd.read_csv("preprocessed_data/claims_db.csv")
major_claims_df = claims_db[claims_db['assertion_type'] == 'major_claim']


In [2]:
authors_db[authors_db["author_key"].str.contains("promislow")]

,author_id,article_id,name,sex,leading_author,first_author,career_stage,expertise_level,author_key
640,659,1669,Promislow DE,Male,True,False,Junior PI,Newcomer,promislow de
699,718,1921,Promislow D,Male,True,False,Junior PI,Newcomer,promislow d
736,716,1848,Promislow DE,Male,True,False,Junior PI,Newcomer,promislow de
746,768,1945,Promislow DE,Male,True,False,Senior PI,Newcomer,promislow de


## B.1. Read the stat author xlsx files
- **0: First** `input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/0_First author cleaned_March 18th.xlsx`
  - Pour les statistiques sur ceux qui deviennent PI, Il ne faut pas prendre en compte ceux qui sont surlignés en bleu car ils sont des premiers auteurs qui sont déjà PI. Normalement ce fichier est bon.  
Avec un peu d’approximation, je trouve que sur 291 first authors: 107 become PI et il y  16  ?? (indéterminés)et 13 qui sont déjà PI en bleu :  ce qui fait 107/262=>40,8% (262=291-16-13)  
- **2: Both** `input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/2_2025_March 9th_stats_author.xlsx`
  - obtenu depuis le site
  -  tu trouveras les last avec de nouveau critères  pour historical (tradition), continuty, first and last.  (pour junior et senior il faut voir par articles dans la database)  
  - Feuille 1 : Parmi les first author (1er feuille) :  
    - Zhenting Zhang et Zhaolin Zhang  sont fusionnés en Zhang Z  
    - Zhipeng Wang et Zhi Wang sont fusionné en Wang Z  
    - Hedengren M et Hedengren M et Hedengren-Olcott M ne sont pas fusionnés
  - Feuille 3: pour les last authors:
    - Il faut fusionner Schneider DS et Schneider D -> c'est les mêmes
    - Historical lab (tradition) or not : il faut pendre le colonne Historical lab after 1998 et comparer 1 (historical, trained in hustorical lab) et 0 (non historical) mais ne pas prendre en compte 2.
    - La colonne E first and Last : compte les chercheurs qui apparaissent comme first author  et last authors. Il faudrait comparer 1 (last author qui ont été first author dans une autre labo mais pas PI)  à  0 (last author withouth being first author)  sans prendre en compte les 2 (first author who were already PI when first author).
  

In [ ]:
# read an xlx file in pandas
input_file_folder = "input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th"

stat_author_fn = f"{input_file_folder}/2_2025_March 9th_stats_author.xlsx"

first_authors_claims = pd.read_excel(stat_author_fn, sheet_name='First')
leading_authors_claims = pd.read_excel(stat_author_fn, sheet_name='Leading')
leading_authors_claims["Authorship"]= "Leading"
first_authors_claims["Authorship"]= "First"

first_authors_claims = clean_authors_claims(first_authors_claims)
leading_authors_claims = clean_authors_claims(leading_authors_claims)

leading_authors_claims = preprocess_utils.build_author_key(leading_authors_claims, author_name_col="Name", key_col="last_author_key")
first_authors_claims = preprocess_utils.build_author_key(first_authors_claims, author_name_col="Name", key_col="first_author_key")

In [4]:
first_authors_claims

,Name,Sex,Articles,Major claims,Unchallenged,Verified,Partially Verified,Mixed,Challenged,Authorship,first_author_key
0,Williams MJ,Male,5,14,7,3,1,0,3,First,williams mj
1,Lemaitre B,Male,4,12,0,11,1,0,0,First,lemaitre b
2,Ha EM,Female,4,10,3,1,2,2,2,First,ha em
3,Leulier F,Male,4,10,0,10,0,0,0,First,leulier f
4,De Gregorio E,Male,3,9,1,8,0,0,0,First,de gregorio e
...,...,...,...,...,...,...,...,...,...,...,...
287,Schlenke TA,Male,1,1,0,1,0,0,0,First,schlenke ta
288,Uttenweiler-Joseph S,Female,1,1,0,1,0,0,0,First,uttenweiler-joseph s
289,Wang LN,Female,1,1,0,1,0,0,0,First,wang ln
290,Yuan Y,Male,1,1,0,1,0,0,0,First,yuan y


In [5]:
leading_authors_claims

,Name,Historical lab,Historical lab after 1998,Continuity,F and L,Sex,Articles,Major claims,Unchallenged,Verified,Partially Verified,Mixed,Challenged,Start lab,Finish,Authorship,last_author_key
0,Hultmark D,True,2.0,True,2.0,Male,28.0,67.0,12.0,47.0,6.0,0.0,2.0,H,Retired,Leading,hultmark d
1,Lemaitre B,True,1.0,True,1.0,Male,21.0,60.0,7.0,47.0,5.0,0.0,1.0,1998,still active,Leading,lemaitre b
2,Hoffmann JA,True,2.0,True,2.0,Male,16.0,45.0,2.0,39.0,2.0,0.0,2.0,H,Retired,Leading,hoffmann ja
4,Lee WJ,False,0.0,True,0.0,Male,12.0,31.0,8.0,14.0,3.0,2.0,4.0,2000,still active,Leading,lee wj
5,Ip YT,False,0.0,False,1.0,Male,11.0,30.0,5.0,21.0,3.0,1.0,0.0,1998,2020,Leading,ip yt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,Nappi AJ,True,2.0,True,0.0,Male,1.0,1.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,Leading,nappi aj
154,Palli SR,False,0.0,False,0.0,Male,1.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,Leading,palli sr
155,Seroude L,False,0.0,False,0.0,Male,1.0,1.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,Leading,seroude l
162,Schneider DS,False,0.0,True,2.0,Male,12.0,31.0,17.0,6.0,1.0,0.0,7.0,NaN,NaN,Leading,schneider ds


In fact, first_authors_claims is useless here as everything is in the `first_authors_covar` file. However, leading_authors_claims has precious information 

In [6]:
first_authors_covar_fn = f"{input_file_folder}/0_First author cleaned_March 18th.xlsx"

highlighted_blue = pd.DataFrame(["Gateff E", "Hultmark D", "Imler JL", "Kappler C", "Kurucz E", 
                                "Lanot R", "Meister M", "Nappi AJ", "Reichhart JM", "RIZKI MT", 
                                "Rizki RM", "Schneider DS", "Wegner KM"], columns=["Name"])
highlighted_blue = preprocess_utils.build_author_key(highlighted_blue, author_name_col="Name", key_col="first_author_key")

first_authors_covar = pd.read_excel(first_authors_covar_fn)
first_authors_covar["Name"] = first_authors_covar["AAA_first author"]
first_authors_covar["Become a Pi"] = first_authors_covar["AAA_Become a Pi"]
first_authors_covar = first_authors_covar.drop(columns=["AAA_first author", "AAA_Become a Pi","Source"])

first_authors_covar = clean_authors_claims(first_authors_covar)
print(first_authors_covar["Sex"].unique(), 
        first_authors_covar["MD"].unique(), 
        first_authors_covar["Become a Pi"].unique(), 
        first_authors_covar["Ivy league"].unique())


# Check for duplicates in Name
duplicated_keys = first_authors_covar[first_authors_covar["Name"].duplicated(keep=False)]
# Sort by Name to see duplicates together
duplicated_keys = duplicated_keys.sort_values("Name")
# Print the result
print(f"Found {len(duplicated_keys)} rows with duplicate keys")
print(duplicated_keys[["Name", "Become a Pi", "Sex"]])

# These are put manually int he xlsx file:
first_authors_covar.loc[262, "Name"] = "Wang Zhipeng"
first_authors_covar.loc[263, "Name"] = "Wang Zhi"
first_authors_covar.loc[285, "Name"] = "Zhang Zhenting"
first_authors_covar.loc[286, "Name"] = "Zhang Zhaolin"
first_authors_covar = preprocess_utils.build_author_key(first_authors_covar, author_name_col="Name", key_col="first_author_key")
first_authors_covar

first_authors_covar = preprocess_utils.build_author_key(first_authors_covar, author_name_col="Name", key_col="first_author_key")
first_authors_covar

first_authors_covar["ignore_for_become_a_pi"] = False
for aut_key in highlighted_blue["first_author_key"]:
    first_authors_covar.loc[first_authors_covar["first_author_key"] == aut_key, "ignore_for_become_a_pi"] = True


# Create a binary column "Is a PI" based on the "PhD Post-doc" column
first_authors_covar["Is a PI"] = first_authors_covar["PhD Post-doc"] == "PI"

first_authors_covar["Career Stage"] = first_authors_covar["PhD Post-doc"].copy()
first_authors_covar["PhD Post-doc"] = first_authors_covar["PhD Post-doc"].map(
    lambda x: "PhD" if x == "PhD" else ("Post-doc" if x == "Post-doc" else pd.NA)
)

first_authors_covar["Become a PI that counts"] = False

first_authors_covar.loc[ 
    (first_authors_covar["Become a Pi"] == True) & 
    (first_authors_covar["ignore_for_become_a_pi"] == False), "Become a PI that counts"] = True
# important that these are NA as they cannot become a PI
first_authors_covar.loc[ 
    (first_authors_covar["Become a Pi"] == True) & 
    (first_authors_covar["ignore_for_become_a_pi"] == True), "Become a PI that counts"] = pd.NA

first_authors_covar

['Male' 'Female' nan] [False nan True] [True False nan] [False True]
Found 4 rows with duplicate keys
        Name Become a Pi     Sex
262   Wang Z       False    Male
263   Wang Z         NaN  Female
285  Zhang Z       False  Female
286  Zhang Z       False  Female


/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_33030/333310170.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df["Ivy league"] == 0, "Ivy league"] = False
/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_33030/1083235099.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  first_authors_covar.loc[


,Sex,PhD Post-doc,current job,MD,Affiliation,Country,Ivy league,Name,Become a Pi,first_author_key,ignore_for_become_a_pi,Is a PI,Career Stage,Become a PI that counts
0,Male,<NA>,PI,False,Forschungszentrum Borstel,Germany,False,Abdelsadik A,True,abdelsadik a,False,False,Senior Staff,True
1,Male,Post-doc,PI,False,Harvard Medical School,USA,True,Agaisse H,True,agaisse h,False,False,Post-doc,True
2,Female,PhD,Industry,False,University of Massachusetts Medical School,USA,False,Aggarwal K,False,aggarwal k,False,False,PhD,False
3,NaN,PhD,Senior Staff,False,niversity of Missouri-Kansas City,USA,False,Ao J,NaN,ao j,False,False,PhD,False
4,Male,Post-doc,PI,False,Harvard Medical School and Massachusetts Gener...,USA,True,Apidianakis Y,True,apidianakis y,False,False,Post-doc,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,Female,PhD,Academia,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,Zhang Zhenting,False,zhang zhenting,False,False,PhD,False
286,Female,PhD,Admin,False,University of Kentucky,USA,False,Zhang Zhaolin,False,zhang zhaolin,False,False,PhD,False
287,Female,PhD,??,False,University of California San Diego,USA,True,Zhao HW,False,zhao hw,False,False,PhD,False
288,Male,PhD,??,False,University of Houston,USA,False,Zhou Z,NaN,zhou z,False,False,PhD,False


## B.2 Build First Author tidy-ish dataframe

author_db that are first_author with first_authors_covar

and _then_ claims with that

In [7]:
len(authors_db[authors_db["first_author"]]["author_key"].unique())

292

In [8]:
first_authors_covar

,Sex,PhD Post-doc,current job,MD,Affiliation,Country,Ivy league,Name,Become a Pi,first_author_key,ignore_for_become_a_pi,Is a PI,Career Stage,Become a PI that counts
0,Male,<NA>,PI,False,Forschungszentrum Borstel,Germany,False,Abdelsadik A,True,abdelsadik a,False,False,Senior Staff,True
1,Male,Post-doc,PI,False,Harvard Medical School,USA,True,Agaisse H,True,agaisse h,False,False,Post-doc,True
2,Female,PhD,Industry,False,University of Massachusetts Medical School,USA,False,Aggarwal K,False,aggarwal k,False,False,PhD,False
3,NaN,PhD,Senior Staff,False,niversity of Missouri-Kansas City,USA,False,Ao J,NaN,ao j,False,False,PhD,False
4,Male,Post-doc,PI,False,Harvard Medical School and Massachusetts Gener...,USA,True,Apidianakis Y,True,apidianakis y,False,False,Post-doc,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,Female,PhD,Academia,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,Zhang Zhenting,False,zhang zhenting,False,False,PhD,False
286,Female,PhD,Admin,False,University of Kentucky,USA,False,Zhang Zhaolin,False,zhang zhaolin,False,False,PhD,False
287,Female,PhD,??,False,University of California San Diego,USA,True,Zhao HW,False,zhao hw,False,False,PhD,False
288,Male,PhD,??,False,University of Houston,USA,False,Zhou Z,NaN,zhou z,False,False,PhD,False


In [9]:
first_authors_db = authors_db[authors_db["first_author"]].copy()
first_author_all = pd.merge(first_authors_db, first_authors_covar, how="left", left_on="author_key", right_on="first_author_key", suffixes=("_db", "_covar"))
print(len(first_authors_db.author_key.unique()), len(first_authors_covar.first_author_key.unique()), len(first_author_all.author_key.unique()))

292 290 292


In [10]:
#first_authors_covar.sort_values("first_author_key").first_author_key.unique()

In [11]:
first_author_all[first_author_all.Name.isna()]

,author_id,article_id,name,sex,leading_author,first_author,career_stage,expertise_level,author_key,Sex,...,Affiliation,Country,Ivy league,Name,Become a Pi,first_author_key,ignore_for_become_a_pi,Is a PI,Career Stage,Become a PI that counts
30,70,2239,Hedengren-Olcott M,Female,False,True,Post-doc,Experienced,hedengren-olcott m,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163,338,2311,Pearson AM,Male,False,True,Post-doc,Experienced,pearson am,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,365,3447,Rizki TM,Male,True,True,Senior PI,Experienced,rizki tm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,363,3448,Rizki TM,Male,True,True,Senior PI,Experienced,rizki tm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,364,3292,Rizki TM,Male,True,True,Senior PI,Experienced,rizki tm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,362,3273,Rizki TM,Male,True,True,Post-doc,Experienced,rizki tm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# merge the claims:
first_author_all = pd.merge(major_claims_df, first_author_all, how="left", left_on="article_id", right_on="article_id", suffixes=("_claim_db", "_aut"))
first_author_all

,id,id.1,content,article_id,rank,pmid,authors_txt,title,pmid_article,volume,...,Affiliation,Country,Ivy league,Name,Become a Pi,first_author_key,ignore_for_become_a_pi,Is a PI,Career Stage,Become a PI that counts
0,577,577,#gene:FBgn0000279 is immune inducible and is p...,3103.0,2,NaN,Tryselius Y;Samakovlis C;Kimbrell DA;Hultmark D,"CecC, a cecropin gene expressed during metamor...",1740152.0,204,...,University of Stockholm,Sweden,False,Tryselius Y,False,tryselius y,False,False,PhD,False
1,5845,5845,Hemocyte proliferation and lamellocyte formati...,2204.0,3,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,...,University of Stockholm,Sweden,False,Zettervall CJ,False,zettervall cj,False,False,PhD,False
2,5841,5841,The Ras/MAPK pathway is a main regulator of th...,2204.0,1,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,...,University of Stockholm,Sweden,False,Zettervall CJ,False,zettervall cj,False,False,PhD,False
3,5843,5843,The JAK/STAT and JNK pathways are major regula...,2204.0,2,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,...,University of Stockholm,Sweden,False,Zettervall CJ,False,zettervall cj,False,False,PhD,False
4,545,545,#gene:FBgn0035976 functions upstream of #gene:...,2520.0,2,NaN,Gottar M;Gobert V;Michel T;Belvin M;Duyk G;Hof...,The Drosophila immune response against Gram-ne...,11912488.0,416,...,CNRS University of Marseille,France,False,Gottar M,False,gottar m,False,False,PhD,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,9082,9082,Ehrlichia chaffeensis establishes a pathogenic...,1593.0,1,NaN,Luce-Fedrow A;Von Ohlen T;Chapes SK,Ehrlichia chaffeensis infections in Drosophila...,19687202.0,77,...,Kansas State University,USA,False,Luce-Fedrow A,False,luce-fedrow a,False,False,PhD,False
1002,9728,9728,B. bassiana inhibits PO activation by preventi...,1532.0,2,NaN,Matskevich AA;Quintin J;Ferrandon D,The Drosophila PRR GNBP3 assembles effector co...,20201042.0,40,...,CNRS-University of Strasbourg,France,False,Matskevich AA,False,matskevich aa,False,False,PhD,False
1003,1769,1769,"A polyclonal antibody against mammalian IRF-1,...",3004.0,2,NaN,Georgel P;Kappler C;Langley E;Gross I;Nicolas ...,Drosophila immunity. A sequence homologous to ...,7537872.0,23,...,CNRS-University of Strasbourg,France,False,Georgel P,True,georgel p,False,False,Post-doc,True
1004,2906,2906,A pathway in addition to Toll regulates degrad...,2874.0,3,NaN,Wu LP;Anderson KV,Regulated nuclear import of Rel proteins in th...,9510254.0,392,...,Memorial Sloan-Kettering Cancer Center and the...,USA,True,Wu LP,True,wu lp,False,False,Post-doc,True


In [13]:
first_author_all["expertise_level"].unique()

array(['Experienced', 'Newcomer'], dtype=object)

In [14]:
first_author_all = first_author_all.apply(preprocess_utils.safe_strip)
# Map assessment types to standardized categories
first_author_all['assessment_type_grouped'] = first_author_all['assessment_type'].apply(plot_info.group_assessment)
first_author_all.to_csv("preprocessed_data/first_author_claims.csv", index=False)

## B.3 Build Last Author tidy-ish dataframe

author_db that are first_author with first_authors_covar

and _then_ claims with that

In [15]:
authors_db

,author_id,article_id,name,sex,leading_author,first_author,career_stage,expertise_level,author_key
0,2,2053,Brun S,Male,False,True,Post-doc,Experienced,brun s
1,3,2053,Lemaitre B,Male,True,False,Junior PI,Experienced,lemaitre b
2,1,1650,Charroux B,Male,False,True,Post-doc,Experienced,charroux b
3,97,2149,Söderhäll K,Male,True,False,Senior PI,Experienced,soderhall k
4,5,2278,Silverman N,Male,False,True,Post-doc,Newcomer,silverman n
...,...,...,...,...,...,...,...,...,...
792,797,2067,Cerenius L,Male,True,False,Senior PI,Experienced,cerenius l
793,798,2122,Higgins DE,Male,True,False,Senior PI,Newcomer,higgins de
794,799,2184,Mengin-Lecreulx D,Male,True,False,Senior PI,Experienced,mengin-lecreulx d
795,800,2130,Moore KJ,Female,True,False,Junior PI,Newcomer,moore kj


In [16]:
leading_authors_covar = leading_authors_claims.drop(columns=plot_info.assessment_columns + ["Articles", "Major claims"])
leading_authors_covar.loc[leading_authors_covar["F and L"] == 0, "F and L"] = False
leading_authors_covar.loc[leading_authors_covar["F and L"] == 1, "F and L"] = True
leading_authors_covar.loc[leading_authors_covar["F and L"] == 2, "F and L"] = pd.NA

# same for Historical lab after 1998
leading_authors_covar.loc[leading_authors_covar["Historical lab after 1998"] == 0, "Historical lab after 1998"] = False
leading_authors_covar.loc[leading_authors_covar["Historical lab after 1998"] == 1, "Historical lab after 1998"] = True
leading_authors_covar.loc[leading_authors_covar["Historical lab after 1998"] == 2, "Historical lab after 1998"] = pd.NA


/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_33030/2082296278.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  leading_authors_covar.loc[leading_authors_covar["F and L"] == 0, "F and L"] = False
/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_33030/2082296278.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  leading_authors_covar.loc[leading_authors_covar["Historical lab after 1998"] == 0, "Historical lab after 1998"] = False


In [17]:
leading_authors_covar.loc[leading_authors_covar["Name"] == "Ehret-Sabatier L", "Continuity"]

76    True
Name: Continuity, dtype: boolean

In [18]:
# Data correction, from bruno on Apri 24th, 2025:
leading_authors_covar.loc[leading_authors_covar["last_author_key"] == "silverman n", "Historical lab after 1998"] = False
#  Bruno : Je considere maintenant les historical lab comme : 
# K .Söderhall, J. Hoffmann, J.-M. Reichhart, D. Hultmark, H. Steiner, TM Rizki, S. Natori, A. Nappi, I Faye and E. Gateff.
update_to_continuity = ["Badenhorst P", "Boccard F", "Fauvarque MO", "Gay NJ"]
for aut_key in update_to_continuity:
    print( leading_authors_covar.loc[leading_authors_covar["Name"] == aut_key, "Continuity"])
    leading_authors_covar.loc[leading_authors_covar["Name"] == aut_key, "Continuity"] = True

leading_authors_covar.loc[leading_authors_covar["Name"] == "Ehret-Sabatier L", "Continuity"] = False
leading_authors_covar.loc[leading_authors_covar["Name"] == "Tatar M", "Continuity"] = False


49    False
Name: Continuity, dtype: boolean
68    False
Name: Continuity, dtype: boolean
29    False
Name: Continuity, dtype: boolean
22    False
Name: Continuity, dtype: boolean


In [19]:
leading_authors_covar

,Name,Historical lab,Historical lab after 1998,Continuity,F and L,Sex,Start lab,Finish,Authorship,last_author_key
0,Hultmark D,True,<NA>,True,<NA>,Male,H,Retired,Leading,hultmark d
1,Lemaitre B,True,True,True,True,Male,1998,still active,Leading,lemaitre b
2,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
4,Lee WJ,False,False,True,False,Male,2000,still active,Leading,lee wj
5,Ip YT,False,False,False,True,Male,1998,2020,Leading,ip yt
...,...,...,...,...,...,...,...,...,...,...
153,Nappi AJ,True,<NA>,True,False,Male,NaN,NaN,Leading,nappi aj
154,Palli SR,False,False,False,False,Male,NaN,NaN,Leading,palli sr
155,Seroude L,False,False,False,False,Male,NaN,NaN,Leading,seroude l
162,Schneider DS,False,False,True,<NA>,Male,NaN,NaN,Leading,schneider ds


In [20]:
leading_authors_covar

,Name,Historical lab,Historical lab after 1998,Continuity,F and L,Sex,Start lab,Finish,Authorship,last_author_key
0,Hultmark D,True,<NA>,True,<NA>,Male,H,Retired,Leading,hultmark d
1,Lemaitre B,True,True,True,True,Male,1998,still active,Leading,lemaitre b
2,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
4,Lee WJ,False,False,True,False,Male,2000,still active,Leading,lee wj
5,Ip YT,False,False,False,True,Male,1998,2020,Leading,ip yt
...,...,...,...,...,...,...,...,...,...,...
153,Nappi AJ,True,<NA>,True,False,Male,NaN,NaN,Leading,nappi aj
154,Palli SR,False,False,False,False,Male,NaN,NaN,Leading,palli sr
155,Seroude L,False,False,False,False,Male,NaN,NaN,Leading,seroude l
162,Schneider DS,False,False,True,<NA>,Male,NaN,NaN,Leading,schneider ds


In [21]:
len(authors_db[authors_db["leading_author"]]["author_key"].unique())

157

In [22]:
leading_authors_covar[leading_authors_covar["last_author_key"].str.contains("promislow")]

,Name,Historical lab,Historical lab after 1998,Continuity,F and L,Sex,Start lab,Finish,Authorship,last_author_key
45,Promislow DE,False,False,False,False,Male,2006,2010,Leading,promislow de
132,Promislow D,False,False,False,False,Male,NaN,NaN,Leading,promislow d


In [23]:
leading_authors_db = authors_db[authors_db["leading_author"]].copy()
leading_author_all = pd.merge(leading_authors_db, leading_authors_covar, how="left", 
                        left_on="author_key", right_on="last_author_key", suffixes=("_db", "_covar"))
print(len(leading_authors_db.author_key.unique()), len(leading_authors_covar.last_author_key.unique()), len(leading_author_all.author_key.unique()))

157 157 157


In [24]:
# correct hoffmann ja as PI instead of Engineer / Technician
leading_author_all.loc[(leading_author_all["author_key"] == "hoffmann ja") & (leading_author_all["career_stage"] == "Engineer / Technician"), "career_stage"] = "Senior PI"

In [25]:
leading_author_all[leading_author_all["author_key"] == "hoffmann ja"]

,author_id,article_id,name,sex,leading_author,first_author,career_stage,expertise_level,author_key,Name,Historical lab,Historical lab after 1998,Continuity,F and L,Sex,Start lab,Finish,Authorship,last_author_key
5,12,3096,Hoffmann JA,Male,True,False,Senior PI,Experienced,hoffmann ja,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
9,22,3015,Hoffmann JA,Male,True,False,Senior PI,Experienced,hoffmann ja,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
15,44,2593,Hoffmann JA,Male,True,False,Senior PI,Experienced,hoffmann ja,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
19,58,3052,Hoffmann JA,Male,True,False,Senior PI,Experienced,hoffmann ja,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
65,141,3141,Hoffmann JA,Male,True,False,Senior PI,Experienced,hoffmann ja,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
66,143,2891,Hoffmann JA,Male,True,False,Senior PI,Experienced,hoffmann ja,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
83,179,3011,Hoffmann JA,Male,True,False,Senior PI,Experienced,hoffmann ja,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
90,195,2945,Hoffmann JA,Male,True,False,Senior PI,Experienced,hoffmann ja,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
91,199,2978,Hoffmann JA,Male,True,False,Senior PI,Experienced,hoffmann ja,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
100,203,2849,Hoffmann JA,Male,True,False,Senior PI,Experienced,hoffmann ja,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja


In [26]:
# merge the claims:
leading_author_all = pd.merge(major_claims_df, leading_author_all, how="left", left_on="article_id", right_on="article_id", suffixes=("_claim_db", "_aut"))
leading_author_all

,id,id.1,content,article_id,rank,pmid,authors_txt,title,pmid_article,volume,...,Name,Historical lab,Historical lab after 1998,Continuity,F and L,Sex,Start lab,Finish,Authorship,last_author_key
0,577,577,#gene:FBgn0000279 is immune inducible and is p...,3103.0,2,NaN,Tryselius Y;Samakovlis C;Kimbrell DA;Hultmark D,"CecC, a cecropin gene expressed during metamor...",1740152.0,204,...,Hultmark D,True,<NA>,True,<NA>,Male,H,Retired,Leading,hultmark d
1,5845,5845,Hemocyte proliferation and lamellocyte formati...,2204.0,3,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,...,Hultmark D,True,<NA>,True,<NA>,Male,H,Retired,Leading,hultmark d
2,5841,5841,The Ras/MAPK pathway is a main regulator of th...,2204.0,1,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,...,Hultmark D,True,<NA>,True,<NA>,Male,H,Retired,Leading,hultmark d
3,5843,5843,The JAK/STAT and JNK pathways are major regula...,2204.0,2,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,...,Hultmark D,True,<NA>,True,<NA>,Male,H,Retired,Leading,hultmark d
4,545,545,#gene:FBgn0035976 functions upstream of #gene:...,2520.0,2,NaN,Gottar M;Gobert V;Michel T;Belvin M;Duyk G;Hof...,The Drosophila immune response against Gram-ne...,11912488.0,416,...,Royet J,True,True,True,False,Male,1999,still active,Leading,royet j
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,9082,9082,Ehrlichia chaffeensis establishes a pathogenic...,1593.0,1,NaN,Luce-Fedrow A;Von Ohlen T;Chapes SK,Ehrlichia chaffeensis infections in Drosophila...,19687202.0,77,...,Chapes SK,False,False,False,False,Male,NaN,NaN,Leading,chapes sk
1002,9728,9728,B. bassiana inhibits PO activation by preventi...,1532.0,2,NaN,Matskevich AA;Quintin J;Ferrandon D,The Drosophila PRR GNBP3 assembles effector co...,20201042.0,40,...,Ferrandon D,True,True,True,True,Male,1999,still active,Leading,ferrandon d
1003,1769,1769,"A polyclonal antibody against mammalian IRF-1,...",3004.0,2,NaN,Georgel P;Kappler C;Langley E;Gross I;Nicolas ...,Drosophila immunity. A sequence homologous to ...,7537872.0,23,...,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
1004,2906,2906,A pathway in addition to Toll regulates degrad...,2874.0,3,NaN,Wu LP;Anderson KV,Regulated nuclear import of Rel proteins in th...,9510254.0,392,...,Anderson KV,False,False,False,False,Female,1997,2010,Leading,anderson kv


In [27]:
leading_author_all["career_stage"].unique()

array(['Senior PI', 'Junior PI', 'Post-doc', 'Master'], dtype=object)

In [28]:
leading_author_all[leading_author_all["author_key"] == "hoffmann ja"]["career_stage"]

35      Senior PI
43      Senior PI
44      Senior PI
48      Senior PI
58      Senior PI
78      Senior PI
79      Senior PI
80      Senior PI
84      Senior PI
184     Senior PI
239     Senior PI
240     Senior PI
261     Senior PI
262     Senior PI
377     Senior PI
378     Senior PI
379     Senior PI
380     Senior PI
381     Senior PI
382     Senior PI
386     Senior PI
388     Senior PI
427     Senior PI
430     Senior PI
431     Senior PI
446     Senior PI
447     Senior PI
448     Senior PI
487     Senior PI
530     Senior PI
531     Senior PI
612     Senior PI
763     Senior PI
880     Senior PI
891     Senior PI
893     Senior PI
898     Senior PI
932     Senior PI
940     Senior PI
941     Senior PI
969     Senior PI
978     Senior PI
980     Senior PI
981     Senior PI
1003    Senior PI
Name: career_stage, dtype: object

In [29]:
leading_author_all["Junior Senior"] = leading_author_all["career_stage"].map(
    lambda x: "Junior PI" if x == "Junior PI" else ("Senior PI" if x == "Senior PI" else pd.NA)
)

# check taht we don't bring back the bug with the bad indexing
assert (leading_author_all[leading_author_all['last_author_key'] == "hoffmann ja"]["Junior Senior"] == leading_author_all[leading_author_all['last_author_key'] == "hoffmann ja"]["career_stage"]).all()

In [30]:
leading_author_all = leading_author_all.apply(preprocess_utils.safe_strip)

# Map assessment types to standardized categories
leading_author_all['assessment_type_grouped'] = leading_author_all['assessment_type'].apply(plot_info.group_assessment)


leading_author_all.to_csv("preprocessed_data/leading_author_claims.csv", index=False)

# Unused stuff

### Altnernative: First author short dataframe Merge author_db.article_id inside the covar xlsx

but not long form... so less accurate

In [31]:
do_alt = False
if do_alt:
    # create an empty str column in first_authors_covar for article_id
    first_authors_covar["article_id"] = ""
    first_authors_covar["Articles"] = 0

    # TODO: check unmatched first authors from authors_db
    for aut_key in first_authors_covar["first_author_key"].unique():
        assert first_authors_covar[first_authors_covar["first_author_key"] == aut_key].shape[0] == 1, f"💥 {aut_key}, {first_authors_covar[first_authors_covar['first_author_key'] == aut_key].shape[0]}"
        matches = authors_db[authors_db["author_key"] == aut_key]
        fh_matches = matches[matches["first_author"]]
        lh_matches = matches[matches["leading_author"]]
        if len(fh_matches) == 0:
            print(f"💥 {aut_key} not found in authors_covar")
            first_authors_covar.loc[first_authors_covar["first_author_key"] == aut_key, "article_id"] = pd.NA
        else:
            # Handle authors with multiple articles by joining them with commas
            article_ids = fh_matches["article_id"].astype(str).tolist()
            article_id_str = ",".join(article_ids)
            first_authors_covar.loc[first_authors_covar["first_author_key"] == aut_key, "article_id"] = article_id_str
            first_authors_covar.loc[first_authors_covar["first_author_key"] == aut_key, "Articles"] = len(article_ids)

In [32]:
if do_alt:
    for aut_key in first_authors_covar["first_author_key"]:
        if len(all_LH[all_LH["first_author_key"] == aut_key]) != 1:
            print (f"outer: 💥 {aut_key}, {len(all_LH[all_LH['first_author_key'] == aut_key])}")
        if len(all_LH_inner[all_LH_inner["first_author_key"] == aut_key]) != 1:
            print (f"inner: 💥 {aut_key}, {len(all_LH_inner[all_LH_inner['first_author_key'] == aut_key])}")

## Summary
So how to analyze ? The key works well to merge at least, so let's have it. I think we should not do by authors because the aurhors are fleeting. Whether it's first or not. Let's do it by claim when it is needed. I think it's much more correct.

In [33]:
assert False, "💥"

AssertionError: 💥

## Deprecated: author_pairs file!


### B.1. Read the Triage last_author files which contains all pairs

update April 7, 2025: we don't care avout this file anymore

In [ ]:
# Read Author info, which contains all the pairs
paper_auth_pairs = pd.read_excel('input_data/2025-02-14_last_xlsx/1_Triage_Last author.xlsx', sheet_name='Tri sans les doublons')
# Drop all columns with 'Unnamed' in the name
paper_auth_pairs = paper_auth_pairs.drop(columns=[col for col in paper_auth_pairs.columns if 'Unnamed' in col]).drop(columns=['Source'])
# rename RIZKI MT	to Rizki TM in column last author
paper_auth_pairs['last author'] = paper_auth_pairs['last author'].replace({'Rizki MT': 'Rizki TM','RIZKI MT': 'Rizki TM' })

paper_auth_pairs.loc[paper_auth_pairs["MD"] == "??", "MD"] = np.nan
paper_auth_pairs.loc[paper_auth_pairs["MD"] == "?", "MD"] = np.nan
paper_auth_pairs.loc[paper_auth_pairs["MD"] == 'O', "MD"] = False
paper_auth_pairs.loc[paper_auth_pairs["MD"] == 0, "MD"] = False
paper_auth_pairs.loc[paper_auth_pairs["MD"] == 1, "MD"] = True

paper_auth_pairs.loc[paper_auth_pairs["Become a Pi"] == "??", "Become a Pi"] = np.nan
paper_auth_pairs.loc[paper_auth_pairs["Become a Pi"] == 0, "Become a Pi"] = False
paper_auth_pairs.loc[paper_auth_pairs["Become a Pi"] == 1, "Become a Pi"] = True

paper_auth_pairs.loc[paper_auth_pairs["Ivy league"] == 0, "Ivy league"] = False
paper_auth_pairs.loc[paper_auth_pairs["Ivy league"] == 1, "Ivy league"] = True

paper_auth_pairs.loc[paper_auth_pairs["Sex"] == "??", "Sex"] = np.nan
paper_auth_pairs.loc[paper_auth_pairs["Sex"] == 1, "Sex"] = "Male"
paper_auth_pairs.loc[paper_auth_pairs["Sex"] == 0, "Sex"] = "Female"

paper_auth_pairs.loc[paper_auth_pairs["current job"] == "pi", "current job"] = "PI"
paper_auth_pairs.loc[paper_auth_pairs["current job"] == "Pi", "current job"] = "PI"
paper_auth_pairs.loc[paper_auth_pairs["current job"] == "?", "current job"] = np.nan
paper_auth_pairs.loc[paper_auth_pairs["current job"] == "??", "current job"] = np.nan

# create merge columns: lowercased and stripped of accents
paper_auth_pairs['first_author_key'] = paper_auth_pairs['first author'].str.lower()
paper_auth_pairs['first_author_key'] = paper_auth_pairs['first_author_key'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
paper_auth_pairs = preprocess_utils.clean_author_keys(paper_auth_pairs, "first_author_key")

paper_auth_pairs.loc[:, 'last_author_key'] = paper_auth_pairs['last author'].str.lower()
paper_auth_pairs.loc[:, 'last_author_key'] = (paper_auth_pairs['last_author_key']
    .str.normalize('NFKD')
    .str.encode('ascii', errors='ignore')
    .str.decode('utf-8'))
#paper_auth_pairs.loc[:, 'last_author_key'] = paper_auth_pairs['last_author_key'].str.replace('rizki mt', 'rizki tm')
paper_auth_pairs = preprocess_utils.clean_author_keys(paper_auth_pairs, "last_author_key")


print(paper_auth_pairs["Sex"].unique(), paper_auth_pairs["MD"].unique(), paper_auth_pairs["Become a Pi"].unique(), paper_auth_pairs["Ivy league"].unique())

paper_auth_pairs = paper_auth_pairs.apply(preprocess_utils.safe_strip)


['Female' 'Male' nan] [False nan True] [True False nan] [True False]


/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_10429/2740770010.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  paper_auth_pairs.loc[paper_auth_pairs["Ivy league"] == 0, "Ivy league"] = False


In [ ]:
paper_auth_pairs

,last author,first author,Sex,PhD Post-doc,Become a Pi,current job,MD,Affiliation,Country,Ivy league,first_author_key,last_author_key
0,Agaisse H,Derré,Female,Post-doc,True,PI,False,Yale University,USA,True,derre i,agaisse h
1,Aguilera RJ,Seong CS,Male,PhD,False,Academia,False,The University of Texas,USA,False,seong cs,aguilera rj
2,Aigaki T,Tsuda M,Male,PhD,True,Admin,False,Tokyo Metropolitan University,Japan,False,tsuda m,aigaki t
3,Ando,Markus,Male,PhD,False,Facility leader,False,"Hungarian Academy of Sciences, Szeged",Hungary,False,markus r,ando i
4,Ando,Rus F,Female,PhD,False,NaN,False,"Hungarian Academy of Sciences, Szeged",Hungary,False,rus f,ando i
...,...,...,...,...,...,...,...,...,...,...,...,...
315,Yu XQ,Ao J,NaN,PhD,NaN,Senior Staff,False,niversity of Missouri-Kansas City,USA,False,ao j,yu xq
316,Zhu S,Yuan Y,Male,PhD,False,Senior Staff,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,yuan y,zhu s
317,Zhu S,Tian C,Female,PhD,True,PI,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,tian c,zhu s
318,Zhu S,Zhang Z,Female,PhD,False,Academia,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,zhang z,zhu s


In [ ]:
paper_auth_pairs

,last author,first author,Sex,PhD Post-doc,Become a Pi,current job,MD,Affiliation,Country,Ivy league,first_author_key,last_author_key
0,Agaisse H,Derré,Female,Post-doc,True,PI,False,Yale University,USA,True,derre i,agaisse h
1,Aguilera RJ,Seong CS,Male,PhD,False,Academia,False,The University of Texas,USA,False,seong cs,aguilera rj
2,Aigaki T,Tsuda M,Male,PhD,True,Admin,False,Tokyo Metropolitan University,Japan,False,tsuda m,aigaki t
3,Ando,Markus,Male,PhD,False,Facility leader,False,"Hungarian Academy of Sciences, Szeged",Hungary,False,markus r,ando i
4,Ando,Rus F,Female,PhD,False,NaN,False,"Hungarian Academy of Sciences, Szeged",Hungary,False,rus f,ando i
...,...,...,...,...,...,...,...,...,...,...,...,...
315,Yu XQ,Ao J,NaN,PhD,NaN,Senior Staff,False,niversity of Missouri-Kansas City,USA,False,ao j,yu xq
316,Zhu S,Yuan Y,Male,PhD,False,Senior Staff,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,yuan y,zhu s
317,Zhu S,Tian C,Female,PhD,True,PI,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,tian c,zhu s
318,Zhu S,Zhang Z,Female,PhD,False,Academia,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,zhang z,zhu s


In [ ]:
paper_auth_pairs["article_id"] = pd.NA
paper_auth_pairs["article_id"] = paper_auth_pairs["article_id"].astype("Int64")

In [ ]:
for art in authors["article_id"].unique():
    these_authors = authors[authors["article_id"] == art]
    these_first_authors = these_authors[these_authors["first_author"] == True]["author_key"].values
    these_leading_authors = these_authors[these_authors["leading_author"] == True]["author_key"].values
    if (len(these_first_authors) == 1 and len(these_leading_authors) == 1):
        indexes = paper_auth_pairs[(paper_auth_pairs["last_author_key"] == these_leading_authors[0]) & 
                                (paper_auth_pairs["first_author_key"] == these_first_authors[0])].index.to_list()
        if len(indexes) == 1:
            paper_auth_pairs.loc[indexes[0], "article_id"] = art
        elif len(indexes) > 1:
            first_na = pd.isna(paper_auth_pairs.loc[indexes, "article_id"]).idxmax() #idmax returns the index of the first occurrence of the maximum value
            print(these_authors["author_key"].values, pd.isna(paper_auth_pairs.loc[indexes, "article_id"]).values, first_na)
            paper_auth_pairs.loc[first_na, "article_id"] = art
        else:
            print(f"NO MATCH 🛑 {art}, {len(indexes)}, {these_first_authors[0]:<20} -- {these_leading_authors[0]:<20} ... {len(these_authors)}")
    else:
        print(f"SKIPPING {art} because only one={len(these_authors)} author: {these_authors['author_key'].values}")

NameError: name 'authors' is not defined

### There is an error above, : 3 hoffman georgel, whereas in the excel there are two...

In [ ]:
paper_auth_pairs[paper_auth_pairs["article_id"].isna()]

In [ ]:
paper_auth_pairs.loc[290, "article_id"] = 2239

In [ ]:
paper_auth_pairs[paper_auth_pairs["article_id"].isna()]

In [ ]:
len(paper_auth_pairs.article_id.unique())

In [ ]:
first_authors_claims

### B.3 Merge to create a covariate file for Last Author

In [ ]:
paper_auth_pairs_LH = paper_auth_pairs[["last author", "last_author_key", "Affiliation", "Country", "Ivy league"]]
paper_auth_pairs_LH = preprocess_utils.deduplicate_by(paper_auth_pairs_LH, "last author").copy()

In [ ]:
 # check for duplicate last_author_key in paper_auth_pairs_LH
paper_auth_pairs_LH[paper_auth_pairs_LH.duplicated('last_author_key', keep=False)]
print(len(paper_auth_pairs_LH[paper_auth_pairs_LH.duplicated('last_author_key', keep=False)]), "duplicates in paper_auth_pairs_LH")
# check for duplicate last_author_key in leading_authors_claims
print(len(leading_authors_claims[leading_authors_claims.duplicated('last_author_key', keep=False)]), "duplicates in leading_authors_claims")

In [ ]:
# Perform the outer merge to see what we are missing
all_LH = pd.merge(leading_authors_claims, paper_auth_pairs_LH, on='last_author_key', how='outer')
print(len(leading_authors_claims), len(paper_auth_pairs_LH), len(all_LH))

In [ ]:
unique_pairs = all_LH[["Name", "last author", "last_author_key"]].drop_duplicates().sort_values("last_author_key", ascending=True)
print(" ONLY in Pairs               only in Claims        matching key ")
print("-"*70)
for i in range(0, len(unique_pairs)):
    if pd.isna(unique_pairs.iloc[i]['last author']) or pd.isna(unique_pairs.iloc[i]['Name']):
        print('💥 ', end='')
        print(f"{unique_pairs.iloc[i]['last author']:<20} vs  {unique_pairs.iloc[i]['Name']:<20} --> {unique_pairs.iloc[i]['last_author_key']:<20}")
    #else:
    #    print(f"   {unique_pairs.iloc[i]['last author']:<20} vs  {unique_pairs.iloc[i]['Name']:<20} --> {unique_pairs.iloc[i]['last_author_key']:<20}")

In [ ]:
all_LH_inner = pd.merge(leading_authors_claims, paper_auth_pairs_LH, on='last_author_key', how='inner').sort_values("last_author_key", ascending=True)
#all_LH_inner.drop(columns=['last_author_key', 'last author', 'Authorship'], inplace=True)
print(len(all_LH_inner))


assessment_columns = ['Unchallenged', 'Verified', 'Partially Verified', 'Mixed', 'Challenged']
# set type of assessment columns to int
for col in assessment_columns:
    all_LH_inner[col] = all_LH_inner[col].astype(int)
all_LH_inner['Major claims'] = all_LH_inner['Major claims'].astype(int)
all_LH_inner['Articles'] = all_LH_inner['Articles'].astype(int)

for col in assessment_columns:
    all_LH_inner[f'{col}_prop'] = all_LH_inner[col] / all_LH_inner['Major claims']

all_LH_inner = all_LH_inner.apply(preprocess_utils.safe_strip)
all_LH_inner.to_csv('preprocessed_data/LH_inner.csv', index=False)
all_LH_inner

In [ ]:
for aut in authors[authors["leading_author"]]["author_key"]:
    if len(all_LH[all_LH["last_author_key"] == aut]) != 1:
        print (f"outer: 💥 {aut}, {len(all_LH[all_LH['last_author_key'] == aut])}")
    if len(all_LH_inner[all_LH_inner["last_author_key"] == aut]) != 1:
        print (f"inner: 💥 {aut}, {len(all_LH_inner[all_LH_inner['last_author_key'] == aut])}")


### B. First authors - deprecated

In [ ]:
all_FH = pd.merge(first_authors_claims, first_authors_claims2, on='first_author_key', how='outer', suffixes=('_claims', '_claims2'))
all_LH_inner = pd.merge(first_authors_claims, first_authors_claims2, on='first_author_key', how='inner').sort_values("first_author_key", ascending=True)
print(len(first_authors_claims), len(first_authors_claims2), len(all_FH), len(all_LH_inner))

In [ ]:
first_authors_claims2.Affiliation.value_counts()

In [ ]:
paper_auth_pairs_FH = paper_auth_pairs.drop(columns=['last author']).sort_values("first author", ascending=True)
paper_auth_pairs_FH

In [ ]:
paper_auth_pairs_FH = preprocess_utils.deduplicate_by(paper_auth_pairs_FH, "first author")

In [ ]:

# check for duplicate first_author_key in paper_auth_pairs_LH
print(len(paper_auth_pairs_FH[paper_auth_pairs_FH.duplicated('first_author_key', keep=False)]), "duplicates in paper_auth_pairs_LH")
# check for duplicate first_author_key in leading_authors_claims
print(len(first_authors_claims[first_authors_claims.duplicated('first_author_key', keep=False)]), "duplicates in leading_authors_claims")


all_FH = pd.merge(first_authors_claims, paper_auth_pairs_FH, on='first_author_key', how='outer', suffixes=('', '_pairs'))
print(len(first_authors_claims), len(paper_auth_pairs_FH), len(all_FH))

print(" ONLY in Pairs               only in Claims        matching key ")
print("-"*70)
unique_pairs = all_FH[["Name", "first author", "first_author_key"]].drop_duplicates().sort_values("first_author_key", ascending=True)
for i in range(0, len(unique_pairs)):
    if pd.isna(unique_pairs.iloc[i]['first author']) or pd.isna(unique_pairs.iloc[i]['Name']):
        print('💥 ', end='')
        print(f" {unique_pairs.iloc[i]['first author']:<20}  {unique_pairs.iloc[i]['Name']:<20} --> {unique_pairs.iloc[i]['first_author_key']:<20}")

In [ ]:


all_FH = all_FH.apply(preprocess_utils.safe_strip)

In [ ]:
all_FH_inner = pd.merge(first_authors_claims, paper_auth_pairs_FH, on='first_author_key', how='inner', suffixes=('', '_pairs')).sort_values("first_author_key", ascending=True)
for idx, row in all_FH_inner.iterrows():
    if row["Sex"] != row["Sex_pairs"]:
        print(f"confligcthing sex for {row['Name']} : {row['Sex']} vs  {row['Sex_pairs']}")
all_FH_inner.drop(columns=["Sex_pairs"], inplace=True)
print(len(all_FH_inner))
all_FH_inner = all_FH_inner.apply(preprocess_utils.safe_strip)

assessment_columns = ['Unchallenged', 'Verified', 'Partially Verified', 'Mixed', 'Challenged']
# set type of assessment columns to int
for col in assessment_columns:
    all_FH_inner[col] = all_FH_inner[col].astype(int)
all_FH_inner['Major claims'] = all_FH_inner['Major claims'].astype(int)
all_FH_inner['Articles'] = all_FH_inner['Articles'].astype(int)

for col in assessment_columns:
    all_FH_inner[f'{col}_prop'] = all_FH_inner[col] / all_FH_inner['Major claims']

all_FH_inner.to_csv('preprocessed_data/FH_inner.csv', index=False)
all_FH_inner

In [ ]:
for aut in authors[authors["first_author"]]["author_key"]:
    #if len(all_FH[all_LH["first_author_key"] == aut]) != 1:
    #    print (f"outer: 💥 {aut}, {len(all_FH[all_FH['author_key'] == aut])}")
    if len(all_FH_inner[all_FH_inner["first_author_key"] == aut]) != 1:
        print (f"inner: 💥 {aut}, {len(all_FH_inner[all_FH_inner['first_author_key'] == aut])}")

In [ ]:
all_FH_inner.columns

In [ ]:
for col in all_FH_inner.columns:
    print(70*"-")
    print(all_FH_inner[col].value_counts())